# 初始化

In [1]:
import os
import time
import cv2
import gc
from keras.applications.imagenet_utils import preprocess_input
import numpy as np  
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras import layers, activations
from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout,ZeroPadding2D,BatchNormalization,Activation,Add,AveragePooling2D,Lambda
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D,MaxPooling2D  

Using TensorFlow backend.


In [2]:
def read_image(image_name):
    im=cv2.imread(image_name)
    im=cv2.resize(im, (224, 224))
    return im

#数据扩充
def img_Rotation(img,angel):
    if(0 == angel):
        dst = img
    else:
        rows,cols=img.shape[:2]
        #angel度旋转
        M=cv2.getRotationMatrix2D((cols/2,rows/2),angel,1)
        dst=cv2.warpAffine(img,M,(cols,rows))
    
    return dst

# 读取训练数据

In [3]:
dataset = 'PatternNet'
base_path='./Dataset/' + dataset
ModelName = "SeResNet50"
batch_size = 20
epochs = 50
Extension='.jpg'

All_Labels = os.listdir(os.path.join(base_path, 'Train'))
num_classes = len(All_Labels)
images_Train = []
labels_Train = []
images_Val = []
labels_Val = []

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Train', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1
            original_img = read_image(finalFileName)
            flipped_img = cv2.flip(original_img, 0)
            
            for j in [0, 45, 90, 135, 180, 225, 270, 315]: 
                images_Train.append(img_Rotation(original_img, j))
                labels_Train.append(Label)               
                images_Train.append(img_Rotation(flipped_img, j))
                labels_Train.append(Label)
                
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " Get %d Train Images" %(len(images_Train)))

2021-11-08 17:29:56 Get 97280 Train Images


In [4]:
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " np.array")
X_Train = np.array(images_Train, dtype='float32')
Y_Train = np.array(labels_Train)
del images_Train
gc.collect()

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " preprocess")
X_Train = preprocess_input(X_Train)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " Done!")

2021-11-08 17:29:59 np.array
2021-11-08 18:19:00 preprocess
2021-11-08 21:55:16 Done!


# SE-ResNet-50

In [5]:
bn_params = {
    'epsilon': 9.999999747378752e-06,
}

def residual_block(input_tensor, filters, reduction=16, strides=1, **kwargs):
    x = input_tensor
    residual = input_tensor

    # bottleneck
    x = Conv2D(filters // 4, (1, 1), kernel_initializer='he_uniform', strides=strides, use_bias=False)(x)
    x = BatchNormalization(**bn_params)(x)
    x = Activation('relu')(x)

    x = ZeroPadding2D(1)(x)
    x = Conv2D(filters // 4, (3, 3), kernel_initializer='he_uniform', use_bias=False)(x)
    x = BatchNormalization(**bn_params)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (1, 1), kernel_initializer='he_uniform', use_bias=False)(x)
    x = BatchNormalization(**bn_params)(x)

    #  if number of filters or spatial dimensions changed
    #  make same manipulations with residual connection
    x_channels = K.int_shape(x)[-1]
    r_channels = K.int_shape(residual)[-1]

    if strides != 1 or x_channels != r_channels:

        residual = Conv2D(x_channels, (1, 1), strides=strides, kernel_initializer='he_uniform', use_bias=False)(residual)
        residual = BatchNormalization(**bn_params)(residual)

    # apply attention module
    x = ChannelSE(x, reduction=reduction)

    # add residual connection
    x = Add()([x, residual])

    x = Activation('relu')(x)

    return x

def ChannelSE(input_tensor, reduction=16):  
    channels = K.int_shape(input_tensor)[-1]
    x = Lambda(lambda a: K.mean(a, axis=[1, 2], keepdims=True))(input_tensor)
    x = Conv2D(channels // reduction, (1, 1), kernel_initializer='he_uniform', activation=activations.relu)(x)
    x = Conv2D(channels, (1, 1), kernel_initializer='he_uniform', activation=activations.hard_sigmoid)(x)
    return layers.Multiply()([input_tensor,x])    #给通道加权重
    
from keras.optimizers import SGD  
def SeResNet(shape):
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
        
    img_input = Input(shape=shape)  

    x = ZeroPadding2D(3)(img_input)
    x = Conv2D(64, (7, 7), strides=2, use_bias=False, kernel_initializer='he_uniform')(x)
    x = BatchNormalization(**bn_params)(x)
    x = Activation('relu')(x)

    x = ZeroPadding2D(1)(x)
    x = MaxPooling2D((3, 3), strides=2)(x)
    
    # body of resnet
    filters = 128
    for i, stage in enumerate([3, 4, 6, 3]):

        # increase number of filters with each stage
        filters *= 2
        reduction = 16

        for j in range(stage):
            # decrease spatial dimensions for each stage (except first, because we have maxpool before)
            if i == 0 and j == 0:
                x = residual_block(x, filters, reduction=reduction, strides=1, is_first=True)
            elif i != 0 and j == 0:
                x = residual_block(x, filters, reduction=reduction, strides=2)
            else:
                x = residual_block(x, filters, reduction=reduction, strides=1)   

    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(256, activation='relu', name='fc256')(x)  
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu', name='fc128')(x)  
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu', name='fc32')(x)  
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax', name='output')(x)  

    model = Model(inputs=img_input,outputs=x, name=ModelName)  
    sgd = SGD(decay=0.0001,momentum=0.9)  
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])  
    
    return model

In [6]:
model = SeResNet(X_Train.shape[1:])
model.load_weights('./models/SeResNet50.h5', by_name=True)
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "SeResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
_______

In [10]:
history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, shuffle=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
97280/97280 [==============================] - 4249s 44ms/step - loss: 1.6057 - accuracy: 0.5122
Epoch 2/50
97280/97280 [==============================] - 4496s 46ms/step - loss: 0.5703 - accuracy: 0.8208
Epoch 3/50
97280/97280 [==============================] - 4328s 44ms/step - loss: 0.3904 - accuracy: 0.8873
Epoch 4/50
97280/97280 [==============================] - 4348s 45ms/step - loss: 0.3299 - accuracy: 0.9073
Epoch 5/50
97280/97280 [==============================] - 4353s 45ms/step - loss: 0.2876 - accuracy: 0.9193
Epoch 6/50
97280/97280 [==============================] - 4764s 49ms/step - loss: 0.2522 - accuracy: 0.9280
Epoch 7/50
97280/97280 [==============================] - 4310s 44ms/step - loss: 0.2472 - accuracy: 0.9299
Epoch 8/50
97280/97280 [==============================] - 4374s 45ms/step - loss: 0.2321 - accuracy: 0.9345
Epoch 9/50
97280/97280 [==================

D:\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.201970). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


97280/97280 [==============================] - 5073s 52ms/step - loss: 0.1721 - accuracy: 0.9525
Epoch 26/50
97280/97280 [==============================] - 4552s 47ms/step - loss: 0.1747 - accuracy: 0.9516
Epoch 27/50
97280/97280 [==============================] - 4790s 49ms/step - loss: 0.1697 - accuracy: 0.9522
Epoch 28/50
97280/97280 [==============================] - 5049s 52ms/step - loss: 0.1668 - accuracy: 0.9542
Epoch 29/50
97280/97280 [==============================] - 4558s 47ms/step - loss: 0.1691 - accuracy: 0.9529
Epoch 30/50
97280/97280 [==============================] - 4587s 47ms/step - loss: 0.1692 - accuracy: 0.9532
Epoch 31/50
97280/97280 [==============================] - 4553s 47ms/step - loss: 0.1619 - accuracy: 0.9547
Epoch 32/50
97280/97280 [==============================] - 4612s 47ms/step - loss: 0.1666 - accuracy: 0.9537
Epoch 33/50
97280/97280 [==============================] - 4715s 48ms/step - loss: 0.1664 - accuracy: 0.9541
Epoch 34/50
97280/97280 [======

In [19]:
import time
save_folder = 'models/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + dataset
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# serialize model to JSON
#import pickle
model_json = model.to_json()
with open(os.path.join(save_folder, ModelName + ".json"), "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save(os.path.join(save_folder, ModelName + ".h5"))
print("Saved Model to disk")

Saved Model to disk


In [ ]:
save_folder = 'models/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + dataset
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " Start to fit...")

for i in range(1, 6):
    history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, shuffle=True)
    model.save(os.path.join(save_folder, ModelName + "_%d.h5" %(i * 10)))
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " Fit %d times..." %(i * 10))
    
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " Done!")

In [ ]:
for i in range(2, 6):
    history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, shuffle=True)
    model.save(os.path.join(save_folder, ModelName + "_%d.h5" %(i * 10)))
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " Fit %d times..." %(i * 10))
    
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " Done!")

In [10]:
images = []
labels = []
outputFileName = []

save_folder2 = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Test', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1    
            images.append(read_image(finalFileName))
            labels.append(Label)
            outputFileName.append(os.path.join(save_folder2, All_Labels[i], file_name).replace(Extension, ".txt"))

print("Get %d Test Images" %(len(images)))
output = np.array(images, dtype="float32")
output = preprocess_input(output)

Get 6080 Test Images


In [ ]:
if not os.path.exists(save_folder2):
    os.makedirs(save_folder2)
    for Label in All_Labels:        
        os.makedirs(os.path.join(save_folder2, Label))
        
from keras.models import Model  
OutPutLayer = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)
print("Saved FeatureMap to disk...")
for i in range(0, len(output)):
    p = OutPutLayer.predict(output[i : i + 1])    
    np.savetxt(outputFileName[i : i + 1][0], p)
    #file_object = open(outputFileName[i : i + 1][0], 'w')
    #file_object.write(str(p))
    #file_object.close()
    print("\r当前输出：%d" %(i + 1), end= " ")

print("\n保存完成！")

In [ ]:
Count = 0
for i in range(0, len(output)):
    p=model.predict(output[i : i + 1])
    if np.argmax(p)==np.argmax(labels[i : i + 1][0]):
        Count += 1
    
print(Count/len(output))

In [13]:
import os
import time
import cv2
from keras.applications.imagenet_utils import preprocess_input

base_path='./Dataset/' + dataset
Extension='.jpg'
All_Labels = os.listdir(os.path.join(base_path, 'Test'))
images = []
labels = []
outputFileName = []

save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Test', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1    
            images.append(read_image(finalFileName))
            labels.append(Label)
            outputFileName.append(os.path.join(save_folder, All_Labels[i], file_name).replace(Extension, ".txt"))

print("Get %d Test Images" %(len(images)))
output = np.array(images, dtype="float32")
output = preprocess_input(output)

Get 6080 Test Images


In [20]:
import os
import time
from keras.models import Model  
OutPutLayer = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)
print("Saved FeatureMap to disk...")
OutputFeatures=[]
for i in range(0, len(output)):
    p = OutPutLayer.predict(output[i : i + 1])
    out=np.reshape(p,p.shape[1])
    OutputFeatures.append(out)
    print("\r当前输出：%d" %(i + 1), end= " ")

OutputFeatures = np.array(OutputFeatures, dtype="float") 
save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
np.savetxt(os.path.join(save_folder, dataset + '-Test-2048D.txt'), OutputFeatures)
print("\n保存完成！")
print(OutputFeatures.shape)

Saved FeatureMap to disk...
当前输出：6080                                                  
保存完成！
(6080, 2048)


In [17]:
K.clear_session()
tf.reset_default_graph()

In [15]:
from keras import layers, activations
from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout,ZeroPadding2D,BatchNormalization,Activation,Add,Dot,AveragePooling2D,Lambda,Reshape
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D,MaxPooling2D,DepthwiseConv2D  

ModelName = "ResNet50"

In [16]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x 
    
from keras.optimizers import SGD  
def ResNet(shape):
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
        
    img_input = Input(shape=shape)  

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = layers.pooling.GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(1024, activation='relu', name='fc1024')(x)  
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', name='fc512')(x)  
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax', name='fc')(x)  

    model = Model(inputs=img_input,outputs=x, name="ResNet50")  
    sgd = SGD(decay=0.0001,momentum=0.9)  
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])  
    
    return model

In [17]:
model = ResNet(X_Train.shape[1:])
model.load_weights('./models/ResNet50.h5', by_name=True)
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, shuffle=True)

Epoch 1/50
97280/97280 [==============================] - 5591s 57ms/step - loss: 0.2624 - accuracy: 0.9320
Epoch 2/50
97280/97280 [==============================] - 5670s 58ms/step - loss: 0.0211 - accuracy: 0.9941
Epoch 3/50
97280/97280 [==============================] - 6139s 63ms/step - loss: 0.0091 - accuracy: 0.9973
Epoch 4/50
97280/97280 [==============================] - 5508s 57ms/step - loss: 0.0046 - accuracy: 0.9987
Epoch 5/50
97280/97280 [==============================] - 5266s 54ms/step - loss: 0.0028 - accuracy: 0.9992
Epoch 6/50
97280/97280 [==============================] - 5363s 55ms/step - loss: 0.0018 - accuracy: 0.9995
Epoch 7/50
97280/97280 [==============================] - 5290s 54ms/step - loss: 0.0013 - accuracy: 0.9996
Epoch 8/50
97280/97280 [==============================] - 5491s 56ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 9/50
97280/97280 [==============================] - 5473s 56ms/step - loss: 8.5031e-04 - accuracy: 0.9998
Epoch 10/50
97280/97280 